In [0]:
import keras
import numpy as np
import pandas as pd
import os

In [0]:
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
os.chdir('drive/My Drive/Colab Notebooks')
!ls

dogs_cats_small  fer2013.csv  fer.ipynb  Untitled0.ipynb  Untitled1.ipynb


In [0]:
def init_weight_and_bias(M1, M2):
    W = np.random.randn(M1, M2) / np.sqrt(M1 + M2)
    b = np.zeros(M2)
    return W.astype(np.float32), b.astype(np.float32)


def sigmoid(A):
    return 1 / (1 + np.exp(-A))


def sigmoid_cost(T, Y):
    return -(T*np.log(Y) + (1-T)*np.log(1-Y)).sum()


def getData(balance_ones=True):
    # images are 48x48 = 2304 size vectors
    Y = []
    X = []
    first = True
    for line in open('fer2013.csv'):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])


    X, Y = np.array(X) / 255.0, np.array(Y)

    if balance_ones:
        # balance the 1 class
        X0, Y0 = X[Y != 1, :], Y[Y != 1]
        X1 = X[Y == 1, :]
        X1 = np.repeat(X1, 9, axis=0)
        X = np.vstack([X0, X1])
        Y = np.concatenate((Y0, [1]*len(X1)))

    return X, Y


def getBinaryData():
    Y = []
    X = []
    first = True
    for line in open('fer2013.csv'):
        if first:
            first = False
        else:
            row = line.split(',')
            y = int(row[0])
            if y == 0 or y == 1:
                Y.append(y)
                X.append([int(p) for p in row[1].split()])
    return np.array(X) / 255.0, np.array(Y)


def error_rate(targets, predictions):
    return np.mean(targets != predictions)

In [8]:
# images are 48x48 = 2304 size vectors
X, Y =getBinaryData()
print(X.shape, Y.shape)

(5500, 2304) (5500,)


In [9]:
N_train = 5000
N_test = 500

Xtrain = X[:N_train, :]
Ytrain = Y[:N_train]
Xtest = X[-N_test:, :]
Ytest = Y[-N_test:]

Xtrain = Xtrain.reshape(N_train, 48, 48, 1)
Xtest = Xtest.reshape(N_test, 48, 48, 1)

print(Xtrain.shape, Ytrain.shape)
print(Xtest.shape, Ytest.shape)

(5000, 48, 48, 1) (5000,)
(500, 48, 48, 1) (500,)


In [0]:
y_train_cat = keras.utils.to_categorical(Ytrain, 7)
y_test_cat = keras.utils.to_categorical(Ytest, 7)
input_shape = (48, 48, 1)

In [0]:
model = keras.models.Sequential()

In [0]:
model = keras.models.Sequential()
model.add(Conv2D(32, kernel_size = (3,3), padding = 'same',
                 input_shape = input_shape, activation = 'relu'))
model.add(MaxPool2D())
model.add(Conv2D(64, kernel_size = (3,3), padding = 'same',
                 activation = 'relu'))
model.add(MaxPool2D())
model.add(Conv2D(128, kernel_size = (3,3), padding = 'same',
                 activation = 'relu'))
model.add(MaxPool2D())
model.add(Conv2D(256, kernel_size = (3,3), padding = 'same',
                 activation = 'relu'))
model.add(MaxPool2D((6,6)))
model.add(Flatten())
model.add(Dense(7, activation = 'softmax'))

In [14]:
model.compile(optimizer=keras.optimizers.Adam(lr = 0.001),
             loss = keras.losses.categorical_crossentropy,
             metrics = ['accuracy'])
model.fit(Xtrain, y_train_cat, batch_size=128, epochs=100)

Epoch 1/100
5000/5000 [==============================] - 2s 353us/step - loss: 0.2782 - acc: 0.9004
Epoch 2/100
5000/5000 [==============================] - 1s 268us/step - loss: 0.2699 - acc: 0.9052
Epoch 3/100
5000/5000 [==============================] - 1s 269us/step - loss: 0.2552 - acc: 0.9054
Epoch 4/100
5000/5000 [==============================] - 1s 267us/step - loss: 0.2581 - acc: 0.9092
Epoch 5/100
5000/5000 [==============================] - 1s 268us/step - loss: 0.2382 - acc: 0.9130
Epoch 6/100
5000/5000 [==============================] - 1s 269us/step - loss: 0.2249 - acc: 0.9182
Epoch 7/100
5000/5000 [==============================] - 1s 269us/step - loss: 0.2097 - acc: 0.9246
Epoch 8/100
5000/5000 [==============================] - 1s 268us/step - loss: 0.1974 - acc: 0.9302
Epoch 9/100
5000/5000 [==============================] - 1s 268us/step - loss: 0.2029 - acc: 0.9240
Epoch 10/100
5000/5000 [==============================] - 1s 266us/step - loss: 0.1893 - acc: 0.9314

In [15]:
score = model.evaluate(Xtest, y_test_cat)
print(score)

500/500 [==============================] - 0s 451us/step
[0.46281298147374766, 0.944]


In [0]:
model_json = model.to_json()

In [0]:
model_json_file = open("model.json", "w")
model_json_file.write(model_json)
model_json_file.close()

In [0]:
model.save_weights('binary_fer_weights.h5')